## Evaluation of reverse geocoding (countries)

First, download [the dataset with locations of countries](https://www.kaggle.com/datasets/liewyousheng/geolocation/code) and put csv files into `datasets/geolocations`

In [1]:
import pandas as pd

from geocoding import settings

df = pd.read_csv(settings.DATASETS_DIR / "geolocations" / "countries.csv")

df.head()

,id,name,iso3,iso2,numeric_code,phone_code,capital,currency,currency_name,currency_symbol,tld,native,region,subregion,timezones,latitude,longitude,emoji,emojiU
0,1,Afghanistan,AFG,AF,4,93,Kabul,AFN,Afghan afghani,؋,.af,افغانستان,Asia,Southern Asia,"[{zoneName:'Asia\/Kabul',gmtOffset:16200,gmtOf...",33.000000,65.0,🇦🇫,U+1F1E6 U+1F1EB
1,2,Aland Islands,ALA,AX,248,+358-18,Mariehamn,EUR,Euro,€,.ax,Åland,Europe,Northern Europe,"[{zoneName:'Europe\/Mariehamn',gmtOffset:7200,...",60.116667,19.9,🇦🇽,U+1F1E6 U+1F1FD
2,3,Albania,ALB,AL,8,355,Tirana,ALL,Albanian lek,Lek,.al,Shqipëria,Europe,Southern Europe,"[{zoneName:'Europe\/Tirane',gmtOffset:3600,gmt...",41.000000,20.0,🇦🇱,U+1F1E6 U+1F1F1
3,4,Algeria,DZA,DZ,12,213,Algiers,DZD,Algerian dinar,دج,.dz,الجزائر,Africa,Northern Africa,"[{zoneName:'Africa\/Algiers',gmtOffset:3600,gm...",28.000000,3.0,🇩🇿,U+1F1E9 U+1F1FF
4,5,American Samoa,ASM,AS,16,+1-684,Pago Pago,USD,US Dollar,$,.as,American Samoa,Oceania,Polynesia,"[{zoneName:'Pacific\/Pago_Pago',gmtOffset:-396...",-14.333333,-170.0,🇦🇸,U+1F1E6 U+1F1F8


Leave only desired columns (coordinates, information about countries)

In [2]:
source_df = df[["latitude", "longitude", "name", "iso3"]].dropna()
source_df.head()

,latitude,longitude,name,iso3
0,33.000000,65.0,Afghanistan,AFG
1,60.116667,19.9,Aland Islands,ALA
2,41.000000,20.0,Albania,ALB
3,28.000000,3.0,Algeria,DZA
4,-14.333333,-170.0,American Samoa,ASM


Extract coordinates into a separate list

In [3]:
points = list(zip(source_df["latitude"], source_df["longitude"]))

Run reverse geocoding for these points

In [4]:
from geocoding.geocoder import get_countries_by_points

countries = get_countries_by_points(points)

2022-05-26 20:00:47.185 | INFO     | geocoding.scylla.session:get_session:21 - Connecting to Scylla cluster: ['127.0.0.1'] (keyspace: geo)
2022-05-26 20:00:47.270 | INFO     | geocoding.scylla.session:get_session:30 - Connected to Scylla cluster.
2022-05-26 20:00:47.270 | DEBUG    | geocoding.scylla.session:get_session:33 - Remote cluster supports shard awareness.


In [5]:
result_df = pd.DataFrame(
    [
        (
            *point,
            c.name if (c := countries[point]) else None,
            c.code if c else None,
        )
        for point in points
    ],
    columns=["latitude", "longitude", "country_name", "country_code"],
)
result_df.head()

,latitude,longitude,country_name,country_code
0,33.000000,65.0,Afghanistan,AFG
1,60.116667,19.9,Åland,ALA
2,41.000000,20.0,Albania,ALB
3,28.000000,3.0,Algeria,DZA
4,-14.333333,-170.0,None,None


Merge two dataset on coordinates

In [6]:
merged_df = pd.merge(
    source_df, result_df, on=["latitude", "longitude"], suffixes=("_source", "_result")
)
merged_df.head()

,latitude,longitude,name,iso3,country_name,country_code
0,33.000000,65.0,Afghanistan,AFG,Afghanistan,AFG
1,60.116667,19.9,Aland Islands,ALA,Åland,ALA
2,41.000000,20.0,Albania,ALB,Albania,ALB
3,28.000000,3.0,Algeria,DZA,Algeria,DZA
4,-14.333333,-170.0,American Samoa,ASM,None,None


Evaluate results

In [7]:
total_num = len(merged_df)
nulls_num = sum(merged_df["country_code"].isnull())
wrong_num = sum(
    (~merged_df["country_code"].isnull())
    & (merged_df["iso3"] != merged_df["country_code"])
)
correct_num = total_num - nulls_num - wrong_num
print(f"Failed (nulls): {nulls_num}/{total_num} ({nulls_num / total_num * 100:.2f}%)")
print(f"Wrong: {wrong_num}/{total_num} ({wrong_num / total_num * 100:.2f}%)")
print(f"Correct: {correct_num}/{total_num} ({correct_num / total_num * 100:.2f}%)")

Failed (nulls): 35/250 (14.00%)
Wrong: 5/250 (2.00%)
Correct: 210/250 (84.00%)


Show points reverse geocoded incorrectly

In [8]:
merged_df[merged_df["iso3"] != merged_df["country_code"]]

,latitude,longitude,name,iso3,country_name,country_code
4,-14.333333,-170.000000,American Samoa,ASM,None,None
7,18.250000,-63.166667,Anguilla,AIA,None,None
8,-74.650000,4.480000,Antarctica,ATA,None,None
16,24.250000,-76.000000,Bahamas The,BHS,None,None
32,-6.000000,71.500000,British Indian Ocean Territory,IOT,None,None
40,16.000000,-24.000000,Cape Verde,CPV,None,None
41,19.500000,-80.500000,Cayman Islands,CYM,None,None
47,-12.500000,96.833333,Cocos (Keeling) Islands,CCK,None,None
49,-12.166667,44.250000,Comoros,COM,None,None
55,21.500000,-80.000000,Cuba,CUB,None,None
